In [2]:
#KB review: Are we able to install those before the notebook is launched?

!pip install --ignore-installed pycaret[full]
!pip install python-dotenv

  Obtaining dependency information for pycaret[full] from https://files.pythonhosted.org/packages/d5/54/d575af389203fc27d6c6cf7d60c4e67fcabfda4bc8e84271c8a396bd4a03/pycaret-3.1.0-py3-none-any.whl.metadata
  Using cached pycaret-3.1.0-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for ipython>=5.5.0 from https://files.pythonhosted.org/packages/ef/02/fc039fca3ec40a00f962eb6e9da45c507334b0650a3cb9facd38d234fb7a/ipython-8.16.1-py3-none-any.whl.metadata
  Using cached ipython-8.16.1-py3-none-any.whl.metadata (5.9 kB)
  Obtaining dependency information for ipywidgets>=7.6.5 from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Using cached ipywidgets-8.1.1-py3-none-any.whl.metadata (2.4 kB)
  Obtaining dependency information for tqdm>=4.62.0 from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-n

In [3]:
#KB review: Are we able to install those before the notebook is launched?
pip install s3fs

Note: you may need to restart the kernel to use updated packages.


In [5]:
# import libraries
import boto3, os
from sagemaker import get_execution_role
import pandas as pd
from dotenv import load_dotenv
from load_data import load_data



my_region = boto3.session.Session().region_name

In [6]:
#KB review: Please add markdowns corresponding to different sections as per https://github.com/crederauk/AWS-MLOps-module_old/issues/7

#KB review: Can we please include the lines below to be a part of the load_data() function and pick up the values from terraform but allow the user to pass their parameters?
load_dotenv(".env")
role = get_execution_role()

data_location_s3 = os.getenv("data_location_s3")
target = os.getenv("target")
data_location = 's3://{}'.format(data_location_s3)
print(data_location)
print(target)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://streaming-data-platform-ml-data/bakerloo.csv
Bakerloo10


In [7]:
data = load_data(data_location)

#KB review: Can we please just call df=load_data(data_location) directly?
df = data.copy()

#KB review: Can this be a part of the load_data function?
df.shape

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


(7127, 5)

In [28]:
df.head()

,Bakerloo,Bakerloo10,dayOfWeek,hour,minute
0,23,24.0,1,13,38
1,23,22.0,1,13,40
2,23,24.0,1,13,41
3,23,24.0,1,13,42
4,23,24.0,1,13,43


In [8]:
#KB review: This is already done above. Please remove
print(df.shape)

#KB review: Can we please wrap the below in a function to just call something like train_test_split(df)?

# Randomly shuffle the DataFrame
df_shuffled = df.sample(frac=1).reset_index(drop=True)

#KB review: Can we please give the user the option to either randomize the split ot pick up the first 80% of the data?
# Sort by day and then pick the first 80% as your test data. 
train_size = int(0.8 * len(df))
train_data = df_shuffled[:train_size]
test_data = df_shuffled[train_size:]
print(train_data.shape, test_data.shape)

(7127, 5)
(5701, 5) (1426, 5)


In [9]:
#KB review: Can we please import it on the top like the others?
from pycaret.regression import *

#KB review: Can we please give the user some explanation on what's happening here and wrap those function calls into one or more function that would just return the best model??
s = setup(data = train_data, target = target, session_id=123)

,Description,Value
0,Session id,123
1,Target,Bakerloo10
2,Target type,Regression
3,Original data shape,"(5701, 5)"
4,Transformed data shape,"(5701, 5)"
5,Transformed train set shape,"(3990, 5)"
6,Transformed test set shape,"(1711, 5)"
7,Numeric features,4
8,Preprocess,True
9,Imputation type,simple


In [10]:
#KB review: Can we please give the user some explanation on what's happening here?
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.7457,1.4424,1.1872,0.9749,0.0745,0.0537,1.2760
lightgbm,Light Gradient Boosting Machine,0.7590,1.4767,1.2018,0.9743,0.0806,0.0559,0.2090
rf,Random Forest Regressor,0.6727,1.4777,1.1985,0.9743,0.0683,0.0408,0.5380
xgboost,Extreme Gradient Boosting,0.7430,1.5420,1.2249,0.9732,0.0732,0.0499,0.2190
et,Extra Trees Regressor,0.6522,1.6832,1.2785,0.9706,0.0728,0.0389,0.4280
gbr,Gradient Boosting Regressor,0.9095,1.9115,1.3727,0.9666,0.0960,0.0706,0.1960
knn,K Neighbors Regressor,0.9428,2.1239,1.4473,0.9628,0.0917,0.0631,0.0310
dt,Decision Tree Regressor,0.7539,2.3591,1.5215,0.9586,0.0866,0.0435,0.0270
lar,Least Angle Regression,1.1694,2.8901,1.6911,0.9493,0.1369,0.0989,0.0230
br,Bayesian Ridge,1.1694,2.8901,1.6911,0.9493,0.1369,0.0989,0.0210


In [13]:
#KB review: Can we please give the user some markdown explanation on what's happening here?
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [14]:
predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.7380,1.1803,1.0864,0.9796,0.0685,0.0528


,Bakerloo,dayOfWeek,hour,minute,Bakerloo10,prediction_label
2676,2,6,4,16,2.0,2.078840
724,7,0,4,42,5.0,4.484022
4595,24,1,10,18,21.0,23.616290
1112,22,2,10,15,24.0,22.783442
4064,19,5,22,0,20.0,21.120159
...,...,...,...,...,...,...
3854,24,1,19,29,23.0,23.729806
4481,27,0,8,2,28.0,27.726726
467,21,6,14,24,21.0,20.457277
3551,7,0,4,39,4.0,4.099245


In [15]:
final_best_model = finalize_model(best)
print(final_best_model)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Bakerloo', 'dayOfWeek', 'hour',
                                             'minute'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('actual_estimator',
                 <catboost.core.CatBoostRegressor object at 0x7fcd2da18af0>)])


In [16]:
#KB review: Can we please give the user some markdown explanation on what's happening here?
final_prediction = predict_model(final_best_model, data=test_data)
final_prediction.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.6571,0.9153,0.9567,0.9834,0.0575,0.0434


,Bakerloo,dayOfWeek,hour,minute,Bakerloo10,prediction_label
5701,23,0,11,31,23.0,22.807420
5702,17,5,22,11,19.0,19.943685
5703,26,1,15,35,23.0,25.517220
5704,28,1,7,17,26.0,26.968544
5705,18,3,22,44,17.0,17.782480


In [17]:
#KB review: Can we please import it on the top like the others?
from pycaret.utils.generic import check_metric

#KB review: All the metrics are given above. Can we please remove this one?
check_metric(final_prediction[target], final_prediction.prediction_label, 'RMSE')

0.9567

In [18]:
#KB review: Can we please import it on the top like the others? Also I think that the comment needs updating
# trying to save the model
from pycaret.regression import save_model

#KB review: Can we please wrap this and the 3 cells below in a function to simply save the model in s3?
save_model(final_best_model, 'final_best_model')


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Bakerloo', 'dayOfWeek', 'hour',
                                              'minute'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('actual_estimator',
                  <catboost.core.CatBoostRegressor object at 0x7fcd2da18af0>)]),
 'final_best_model.pkl')

In [22]:
#KB review: Can we please import it on the top like the others?
import tarfile

with tarfile.open('final_best_model.tar.gz', 'w:gz') as tar:
    tar.add('final_best_model.pkl')

In [23]:
#KB review: Can we please import it on the top like the others?
import boto3

s3 = boto3.client('s3')

#KB review: The names here should be passed from terraform with var.model_name and var.data_location_s3
s3.upload_file('final_best_model.tar.gz', 'lanre-test-model', 'final_best_model.tar.gz')


In [27]:
#KB review: Can we please wrap this cell into a single function e.g deploy_model_to_endpoint?

#KB review: Can we please import it on the top like the others?
from sagemaker.model import Model

#KB review: The names here should be passed from terraform with var.model_name and var.data_location_s3
model_data = 's3://lanre-test-model/final_best_model.tar.gz'
# Move model to s3. Member must satisfy regular expression pattern: ^(https|s3)://([^/]+)/?(.*)$
model = Model(
    #KB review: Souldn't the image_uri come from terraform instead of being hardcoded?
    image_uri='135544376709.dkr.ecr.eu-west-1.amazonaws.com/mlops-pycaret-repo:latest',  # The ECR image you pushed
    model_data=model_data,  # Location of your serialized model
    role=role
)

predictor = model.deploy(
    initial_instance_count=1,
    #KB review: instance_type should come from the var.endpoint_instance_type (it seems to have been deleted in this MR)
    instance_type='ml.m4.xlarge',
    
    #KB review: This should be var.model_name+"-endpoint"
    endpoint_name="lanre-trial-endpoint"
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
------!

#KB review: This can be done at later stage from a CI/CD pipeline. I'll raise a ticket for it
- Add a shell script to automatically delete the endpoints from sagemaker using a destroy. 
- Import the endpoint before destroy (may fail if endpoint doesnt exist). Use a try and except statement to deal with this.
- 